### Идея метода

0. Случайно выставляем точки в пространстве.
1. Кластеризуем точки. На данном шаге вычисляются все попарные расстояния между центрами кластеров и точками в датасете. Функция расстояния - Евклидова (корень из суммы квадратов) $\sqrt{\sum_{i=1}^N{(center_i-P_i)^2}}$. Для каждой точки в датасете присваивается номер кластера, к которому она принадлежит.
2. Если на данном этапе количество итераций больше либо равно максимальному числу итераций, возвращаем результат кластеризации (то есть наши предсказания классов), иначе повышаем `n_iter += 1`.
3. Пересчитываем центры кластеров. Для каждой координаты высчитываем взвешенную сумму координат тех точек, которые попали в данный кластер. Мы берем среднее для каждой из координат среди всех тех точек, которые попали в кластер. Если для всех центров кластеров их координаты изменились незначительно, прерываем вычисления (алгоритм сошелся), возвращаем результат. Иначе переходим к шагу 1.

# Метод K-Means
- Как работает: сначала выбираются несколько центров для групп (например, для трех групп — три центра).
- Затем алгоритм распределяет все элементы по группам, определяя, к какому центру они ближе.
- После этого он пересчитывает центры для новых групп и повторяет процесс, пока центры не перестанут сильно меняться. Это позволяет создать группы с элементами, схожими между собой.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans as SklearnKMeans
from sklearn.preprocessing import StandardScaler

class SimpleKMeans:
    def __init__(self, n_clusters=3, max_iter=100, random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.inertia_ = None
        self.random_state = random_state
        np.random.seed(random_state)
        
    def fit(self, X):
        self.centroids = X[np.random.choice(X.shape[0], self.n_clusters, replace=False)]
        
        for _ in range(self.max_iter):
            distances = np.sqrt(((X[:, np.newaxis] - self.centroids)**2).sum(axis=2))
            
            # для каждой точки находит индекс ближайшего центроида
            self.labels_ = np.argmin(distances, axis=1) 
            
            # Обновление центроидов
            new_centroids = []
            for k in range(self.n_clusters):
                cluster_points = X[self.labels_ == k]
                if len(cluster_points) > 0:
                    new_centroids.append(cluster_points.mean(axis=0))
                else:
                    new_centroids.append(X[np.random.randint(X.shape[0])])
            
            if np.allclose(self.centroids, new_centroids, atol=1e-4):
                break
                
            self.centroids = np.array(new_centroids)
        self.inertia_ = sum(np.min(distances, axis=1)**2)
            
        return self
